## Example of Identifying Reference Object in Scene

Source: https://dev.to/erol/object-detection-with-color-knl

In [77]:
import cv2
import numpy as np
from collections import deque

In [2]:
buffer_size = 16
pts = deque(maxlen=buffer_size)

In [78]:
# Using HSV: Hue, Saturation and Value for representing color in images to then detect
# Define upper and lower thresholds for the reference object in hsv
# ref_lower = (0, 0, 150)
# ref_upper = (255, 20, 255)

ref_lower = (0, 0, 0)
ref_upper = (255, 70, 30)

In [79]:
# Read image
image = cv2.imread("C:/Users/jzapu/Image_Scaling/blackout.jpg")
# imaget = cv2.imread("C:/Users/jzapu/Image_Scaling/img2.jpg")

In [66]:
new = cv2.putText(imaget, "HELLO", (25, 50), cv2.FONT_HERSHEY_COMPLEX_SMALL, 1, (255, 0, 0), 3)
# cv2.imwrite("output.jpg", out)
cv2.imshow("TEST", new)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [3]:
# See
cv2.imshow("Test Image", image)
cv2.waitKey(0)
cv2.destroyAllWindows()

True

In [80]:
# We will be applying gaussian smoothing, then HSV conversion, then opening (Erosian -> Dilation) to preprocess image for targeting
# Gaussian Operation
blur_image = cv2.GaussianBlur(image, (11, 11), 0)
# RGB -> HSV
hsv = cv2.cvtColor(blur_image, cv2.COLOR_BGR2HSV)
# Morphology: Opening Image
# Define Mask for Operation
mask = cv2.inRange(hsv, ref_lower, ref_upper)
mask = cv2.erode(mask, None, iterations=2)
mask = cv2.dilate(mask, None, iterations=2)

In [81]:
# Now noise is gone: find edges of reference object (all edges)
contours,_ = cv2.findContours(mask.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
center = None

In [49]:
# Find coordinates in image to draw the target shape outline
if (len(contours) > 0):
    target = max(contours, key=cv2.contourArea)
    # (x,y), radius = cv2.minEnclosingCircle(target)
    rect = cv2.minAreaRect(target)
    # center = (int(x),int(y))
    # radius = int(radius)
    ((x,y), (width, height), rotation) = rect
    s = f"x {np.round(x)}, y: {np.round(y)}, width: {np.round(width)}, height: {np.round(height)}, rotation: {np.round(rotation)}"
    box = cv2.boxPoints(rect)
    box = np.int64(box)
    M = cv2.moments(target)
    center = (int(M["m10"] / M["m00"]), int(M["m01"] / M["m00"]))
    # out = cv2.circle(image, center, radius, (0, 255, 0), 4)
    cv2.circle(image, center, 5, (255, 0, 255), -1)
    cv2.drawContours(image, [box], 0, (255, 0, 0), 4)
    fin =  cv2.putText(image, s, (25, 50), cv2.FONT_HERSHEY_COMPLEX_SMALL, 2, (200, 0, 50), 2)
    cv2.imwrite('testout.jpg', fin)
    cv2.imshow("TEST", fin)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    print(center, rect)
else:
    print("Error with thresholds")

(3281, 2152) ((3031.69384765625, 2063.304931640625), (2013.5504150390625, 681.5897827148438), 1.4714945554733276)


In [82]:
num_windows = 0
rect_arr = []
box_arr = []
for i in range(len(contours)):
    target = max(contours, key=cv2.contourArea)
    num_windows = num_windows + 1
    rect = cv2.minAreaRect(contours[i])
    rect_arr.append(rect)
    ((x,y), (width, height), rotation) = rect
    box = cv2.boxPoints(rect)
    box = np.int64(box)
    box_arr.append(box)
    M = cv2.moments(target)
    center = (int(M["m10"] / M["m00"]), int(M["m01"] / M["m00"]))
    #cv2.circle(image, center, 5, (255, 0, 255), -1)
    cv2.drawContours(image, [box], 0, (255, 0, 0), 4)


In [83]:
# Define Wight and Height of the reference in pixels
refWidth = 3.0
refHeight = 4.0

# Width and Height Ratios from last time - NOT ACCURATE HERE WILL NEED TO CAHNGE LATER
widthRatio = 0.003
heightRatio = 0.0063

In [84]:
# Get Total length and width of the side
#############################################
# Dims of image
rows, cols, ch = image.shape
sideHeight = heightRatio * cols
sideWidth = widthRatio * rows
print(sideWidth, sideHeight)

# Draw Lines
image = cv2.line(image, (0, 0), (0, 500), (0, 255, 255), 2)
image = cv2.line(image, (0, 0), (500, 0), (0, 255, 255), 2)

1.5 3.15


In [85]:
# Get end-of-image to first / last window distance
# First Window:
print(num_windows)
print(box_arr[0])
print(box_arr[3])
left_padding = box_arr[0][0][0] - 0
# Last Window
right_padding = cols - box_arr[2][1][0] 
# NOTE: Looks like cv2 going CCW with the box points 
print(left_padding, right_padding)

# Draw Lines
left_start_point = (0, box_arr[0][0][1])
left_end_point = (box_arr[0][0][0], box_arr[0][0][1])
right_start_point = (cols, box_arr[2][1][1])
right_end_point = (box_arr[2][1][0], box_arr[2][1][1])
image = cv2.line(image, left_start_point, left_end_point, (0, 255, 255), 2)
image = cv2.line(image, right_start_point, right_end_point, (0, 255, 255), 2)

7
[[ 43 443]
 [ 47 443]
 [ 47 449]
 [ 43 449]]
[[363 267]
 [449 265]
 [454 456]
 [368 458]]
43 47


In [86]:
# Get Top - bottom distance (top / bottom of image - to - window blackout)
bottom_padding = cols - box_arr[0][2][1]
top_padding = box_arr[0][2][1] - 0

# Draw Lines

image = cv2.line(image, (box_arr[3][0][0], 0), (box_arr[3][0][0], box_arr[3][0][1]), (0, 255, 255), 2)
image = cv2.line(image, (box_arr[3][0][0], box_arr[3][3][1]), (box_arr[3][0][0], cols), (0, 255, 255), 2)

In [87]:
# Get Distance between windows 
# MAP
# arr[4] -> 1
# arr[6] -> 2
# arr[3] -> 3

d1 = box_arr[6][0][0] - box_arr[4][1][0]
d2 = box_arr[3][0][0] - box_arr[6][1][0]
image = cv2.line(image, (box_arr[4][1][0], box_arr[4][1][1]), (box_arr[6][0][0], box_arr[6][0][1]), (0, 255, 255), 2)
image = cv2.line(image, (box_arr[6][1][0], box_arr[6][1][1]), (box_arr[3][0][0], box_arr[3][0][1]), (0, 255, 255), 2)

In [88]:
# SHow and Save
cv2.imshow("TEST", image)
cv2.waitKey(0)
cv2.destroyAllWindows()
#cv2.imwrite('LinesDrawn.jpg', image)

In [89]:
cv2.imshow('Windows Space Determined', image)
cv2.waitKey(0)
cv2.destroyAllWindows()
cv2.imwrite('windowOut.jpg', image)

True

In [74]:
print(box_arr[0])
print(box_arr[1])
print(box_arr[2])
print(box_arr[3])
print(box_arr[4])
print(box_arr[5])
print(box_arr[6])

[[ 43 443]
 [ 47 443]
 [ 47 449]
 [ 43 449]]
[[ 43 383]
 [ 47 383]
 [ 47 388]
 [ 43 388]]
[[448 270]
 [453 270]
 [453 274]
 [448 274]]
[[363 267]
 [449 265]
 [454 456]
 [368 458]]
[[ 47 261]
 [132 261]
 [132 458]
 [ 47 458]]
[[439 260]
 [443 260]
 [443 264]
 [439 264]]
[[207 255]
 [295 255]
 [295 452]
 [207 452]]


In [8]:
# Perspective transform
# NOTE: MAY WANT TO ADD A ROTATION IF ON A SLANT LIKE IN THIS IMAGE

# Dims of image
rows, cols, ch = image.shape

print (rows, cols)


3770 2501


In [10]:
# Get feet / pixel value for reference - use for getting full image 
pwidth = rect[1][0]
pheight = rect[1][1]
# Calculate the Feet / Pixel
wratio = refWidth / pwidth
hratio = refHeight / pheight

print(wratio, hratio)

0.0030110388831515737 0.0062926639901208555


In [11]:
# Output length and width of wall
sideHeight = hratio * cols
sideWidth = wratio * rows
print(sideWidth, sideHeight)

11.351616589481432 15.73795263929226


In [14]:
print(box)

[[1376 2434]
 [2011 2413]
 [2045 3408]
 [1409 3430]]


In [16]:
# Perspective Transform 
# 2DBox starts from bottom right corner and goes clockwise
ppts = np.float32(box)
pts = np.float32([[refWidth, 0], [0, 0], [0, refHeight], [refWidth, refHeight]])
M = cv2.getPerspectiveTransform(ppts, pts)
print(M)

[[-4.72887730e-03  1.61589777e-04  9.11985612e+00]
 [ 1.33161157e-04  4.02653976e-03 -9.98382753e+00]
 [-1.47211344e-06  1.68547735e-06  1.00000000e+00]]


In [20]:
out = cv2.warpPerspective(image, M, (refWidth, refHeight))

error: OpenCV(4.5.3) :-1: error: (-5:Bad argument) in function 'warpPerspective'
> Overload resolution failed:
>  - Can't parse 'dsize'. Sequence item with index 0 has a wrong type
>  - Can't parse 'dsize'. Sequence item with index 0 has a wrong type


In [19]:
cv2.imshow("HERE", out)
cv2.waitKey(0)

-1